In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('INTC.csv')

In [3]:
df90 = df[0:90]
print('90 day high = ',max(df90['High Price']))
print('90 day low = ',min(df90['Low Price']))
print('90 day mean = ',np.mean(df90['Close Price']))

90 day high =  53.5
90 day low =  44.8
90 day mean =  49.44277777777777


In [4]:
df['Date'] = df['Date'].astype('datetime64')
print('Max Date - Min Date = ',max(df['Date']) - min(df['Date']))

Max Date - Min Date =  729 days 00:00:00


In [5]:
df['Month'] = df['Date'].apply(lambda x : x.month)
df['Year'] = df['Date'].apply(lambda x : x.year)
df['Week'] = df['Date'].apply(lambda x : x.week)
df['Total Traded Quantity'] = df['Total Traded Quantity'].str.replace('M', '').astype('float64').apply(lambda x: x * 1000000)
df['Price_Vol'] = df['Total Traded Quantity'] * df['Close Price']

In [6]:
grouped_year = df.groupby(['Year'], sort=False)
grouped_year_month = df.groupby(['Year', 'Month'], sort=False)
grouped_year_month_week = df.groupby(['Year','Month','Week'], sort=False)

In [7]:
VWAP = (grouped_year_month['Price_Vol'].sum())/(grouped_year_month['Total Traded Quantity'].sum())
print(VWAP)

Year  Month
2019  10       51.306537
      9        51.021127
      8        46.742156
      7        50.157748
      6        46.440543
      5        46.350809
      4        55.300776
      3        53.617573
      2        50.757402
      1        47.693553
2018  12       46.841783
      11       47.965452
      10       45.769616
      9        46.311160
      8        48.351875
      7        50.513742
      6        53.288040
      5        54.196260
      4        51.260887
      3        50.861458
      2        45.965462
      1        45.668002
2017  12       45.159359
      11       45.657492
      10       43.778244
dtype: float64


In [8]:
def N_day_average(df, N):
  return np.mean(df[0:N]['Close Price'])

print(N_day_average(df, 50))

50.0268


In [9]:
def per_change_last_N_days(df, N):
  return (df['Close Price'][0] - df['Close Price'][N - 1]) * 100 / (df['Close Price'][N - 1])

print(per_change_last_N_days(df, 50),"%")

14.288840262582044 %


In [10]:
def get_nearest_date_less_than(df, t_date):
  for d in df['Date']:
    if d <= t_date:
      return d

In [11]:
def print_percent_change_std_interval(df):
  curr_date = df['Date'][0]
  td_1w = curr_date - timedelta(days=int(grouped_year_month_week['Date'].count()[0]))
  td_2w = curr_date - timedelta(days=int(grouped_year_month_week['Date'].count()[0] + grouped_year_month_week['Date'].count()[1]))
  td_1m = curr_date - timedelta(days=int(grouped_year_month['Date'].count()[0]))
  td_3m = curr_date - timedelta(days=int(grouped_year_month['Date'].count()[0] 
                                        + grouped_year_month['Date'].count()[1] 
                                        + grouped_year_month['Date'].count()[2]
                                        ))
  td_6m = curr_date - timedelta(days=int(grouped_year_month['Date'].count()[0] 
                                        + grouped_year_month['Date'].count()[1] 
                                        + grouped_year_month['Date'].count()[2]
                                        + grouped_year_month['Date'].count()[3] 
                                        + grouped_year_month['Date'].count()[4] 
                                        + grouped_year_month['Date'].count()[5]
                                        ))
  td_1y = curr_date - timedelta(days=int(list(grouped_year['Date'].count())[0]))

  td_1w_n = get_nearest_date_less_than(df, td_1w)
  td_2w_n = get_nearest_date_less_than(df, td_2w)
  td_1m_n = get_nearest_date_less_than(df, td_1m)
  td_3m_n = get_nearest_date_less_than(df, td_3m)
  td_6m_n = get_nearest_date_less_than(df, td_6m)
  td_1y_n = get_nearest_date_less_than(df, td_1y)

  change_1w = float(df['Close Price'][0] - df[df['Date'] == td_1w_n]['Close Price']) * 100/ float(df[df['Date'] == td_1w_n]['Close Price'])
  change_2w = float(df['Close Price'][0] - df[df['Date'] == td_2w_n]['Close Price']) * 100/ float(df[df['Date'] == td_2w_n]['Close Price'])
  change_1m = float(df['Close Price'][0] - df[df['Date'] == td_1m_n]['Close Price']) * 100/ float(df[df['Date'] == td_1m_n]['Close Price'])
  change_3m = float(df['Close Price'][0] - df[df['Date'] == td_3m_n]['Close Price']) * 100/ float(df[df['Date'] == td_3m_n]['Close Price'])
  change_6m = float(df['Close Price'][0] - df[df['Date'] == td_6m_n]['Close Price']) * 100/ float(df[df['Date'] == td_6m_n]['Close Price'])
  change_1y = float(df['Close Price'][0] - df[df['Date'] == td_1y_n]['Close Price']) * 100/ float(df[df['Date'] == td_1y_n]['Close Price'])

  print("Time Duration     % Change (Return)")
  print("  1 Week        ",change_1w,"%")
  print("  2 Weeks       ",change_2w,"%")
  print("  1 Month       ",change_1m,"%")
  print("  3 Months      ",change_3m,"%")
  print("  6 Months      ",change_6m,"%")
  print("  1 Year        ",change_1y,"%")

In [12]:
print_percent_change_std_interval(df)

Time Duration     % Change (Return)
  1 Week         1.693925233644855 %
  2 Weeks        -0.797720797720801 %
  1 Month        2.5726630007855364 %
  3 Months       16.169928825622765 %
  6 Months       10.050568900126414 %
  1 Year         -4.182718767198681 %


In [13]:
df_filled = df["Close Price"]
df_filled = df_filled[::-1].pct_change().fillna(0)[::-1]
print(df_filled)

0      0.009861
1     -0.005576
2     -0.002302
3      0.014992
4     -0.009641
         ...   
498    0.025242
499   -0.000676
500    0.073761
501    0.013977
502    0.000000
Name: Close Price, Length: 503, dtype: float64


In [14]:
df['day_perc_change'] = df_filled*100
print(df.head())

        Date  Close Price  Open Price  ...  Week     Price_Vol  day_perc_change
0 2019-10-24        52.23       52.11  ...    43  1.339700e+09         0.986079
1 2019-10-23        51.72       51.37  ...    43  8.254512e+08        -0.557585
2 2019-10-22        52.01       51.90  ...    43  6.943335e+08        -0.230194
3 2019-10-21        52.13       51.52  ...    43  6.839456e+08         1.499221
4 2019-10-18        51.36       51.75  ...    42  8.720928e+08        -0.964134

[5 rows x 11 columns]


In [15]:
def add_trend_to_df(val):
  res = ''
  if(val < 0.5 and val>=-0.5):
    res='Slight or no change'
  elif(val < 1 and val>=0.5):
    res='Slight positive'
  elif(val < -0.5 and val>=-1):
    res='Slight negative'
  elif(val < 3 and val>=1):
    res='Positive'
  elif(val < -1 and val>=-3):
    res='Nositive'
  elif(val < 7 and val>=3):
    res='Among top gainers'
  elif(val < -3 and val>=-7):
    res='Among top losers'
  elif(val > 7):
    res='Bull run'
  else:
    res='Bear drop'
  return res

df['Trend'] = df['day_perc_change'].apply(lambda val: add_trend_to_df(val))
print(df.tail())

          Date  Close Price  ...  day_perc_change                Trend
498 2017-10-31        45.49  ...         2.524228             Positive
499 2017-10-30        44.37  ...        -0.067568  Slight or no change
500 2017-10-27        44.40  ...         7.376058             Bull run
501 2017-10-26        41.35  ...         1.397744             Positive
502 2017-10-25        40.78  ...         0.000000  Slight or no change

[5 rows x 12 columns]


In [17]:
print("Mean Volume = ", (df.groupby(df['Trend'])['Total Traded Quantity'].mean()))
print("Median Volume = ",(df.groupby(df['Trend'])['Total Traded Quantity'].median()))

Mean Volume =  Trend
Among top gainers      3.792320e+07
Among top losers       4.548476e+07
Bear drop              7.009000e+07
Bull run               8.847500e+07
Nositive               2.907931e+07
Positive               2.531466e+07
Slight negative        2.282000e+07
Slight or no change    2.114783e+07
Slight positive        2.502393e+07
Name: Total Traded Quantity, dtype: float64
Median Volume =  Trend
Among top gainers      38790000.0
Among top losers       39090000.0
Bear drop              70090000.0
Bull run               88475000.0
Nositive               27040000.0
Positive               23355000.0
Slight negative        19600000.0
Slight or no change    20460000.0
Slight positive        23150000.0
Name: Total Traded Quantity, dtype: float64


In [18]:
df.to_csv('week2.csv')